# Pengelas Imej Mudah

Notebook ini menunjukkan cara untuk mengelas imej menggunakan rangkaian neural yang telah dilatih terlebih dahulu.

**Apa yang akan anda pelajari:**
- Cara memuat dan menggunakan model yang telah dilatih
- Prapemprosesan imej
- Membuat ramalan pada imej
- Memahami skor keyakinan

**Kes penggunaan:** Mengenal pasti objek dalam imej (seperti "kucing", "anjing", "kereta", dll.)

---


## Langkah 1: Import Perpustakaan yang Diperlukan

Mari kita import alat yang diperlukan. Jangan risau jika anda belum memahami semuanya!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Langkah 2: Muatkan Model Pra-latih

Kita akan menggunakan **MobileNetV2**, sebuah rangkaian neural yang telah dilatih dengan berjuta-juta imej.

Ini dipanggil **Pembelajaran Pemindahan** - menggunakan model yang telah dilatih oleh orang lain!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Langkah 3: Fungsi Pembantu

Mari kita cipta fungsi untuk memuat dan menyediakan imej untuk model kita.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Langkah 4: Uji pada Imej Contoh

Mari cuba klasifikasikan beberapa imej dari internet!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Klasifikasikan Setiap Imej


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Langkah 5: Cuba Imej Anda Sendiri!

Gantikan URL di bawah dengan mana-mana URL imej yang anda ingin klasifikasikan.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Apa Yang Baru Saja Berlaku?

1. **Kami memuatkan model yang telah dilatih** - MobileNetV2 telah dilatih menggunakan jutaan imej
2. **Kami memproses imej** - Menukar saiz dan format imej untuk model
3. **Model membuat ramalan** - Ia menghasilkan kebarangkalian untuk 1000 kelas objek
4. **Kami menafsirkan hasil** - Menukar nombor kepada label yang mudah difahami

### Memahami Skor Keyakinan

- **90-100%**: Sangat yakin (hampir pasti betul)
- **70-90%**: Yakin (mungkin betul)
- **50-70%**: Agak yakin (mungkin betul)
- **Di bawah 50%**: Tidak begitu yakin (tidak pasti)

### Kenapa ramalan mungkin salah?

- **Sudut atau pencahayaan yang luar biasa** - Model dilatih menggunakan foto biasa
- **Objek berganda** - Model mengandaikan satu objek utama
- **Objek yang jarang ditemui** - Model hanya mengenali 1000 kategori
- **Imej berkualiti rendah** - Imej kabur atau berpixel sukar untuk dikenali

---


## 🚀 Langkah Seterusnya

1. **Cuba imej yang berbeza:**
   - Cari imej di [Unsplash](https://unsplash.com)
   - Klik kanan → "Salin alamat imej" untuk mendapatkan URL

2. **Bereksperimen:**
   - Apa yang berlaku dengan seni abstrak?
   - Bolehkah ia mengenali objek dari sudut berbeza?
   - Bagaimana ia mengendalikan pelbagai objek?

3. **Belajar lebih lanjut:**
   - Terokai [pelajaran Penglihatan Komputer](../lessons/4-ComputerVision/README.md)
   - Belajar melatih pengklasifikasi imej anda sendiri
   - Fahami bagaimana CNNs (Convolutional Neural Networks) berfungsi

---

## 🎉 Tahniah!

Anda baru sahaja membina pengklasifikasi imej menggunakan rangkaian neural terkini!

Teknik yang sama ini digunakan untuk:
- Google Photos (mengatur foto anda)
- Kereta pandu sendiri (mengenali objek)
- Diagnosis perubatan (menganalisis X-ray)
- Kawalan kualiti (mengesan kecacatan)

Teruskan meneroka dan belajar! 🚀



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
